# vbase
vbases are originally set to 0.

In `as_define_region`, they get assigned some interesting values:
* vaddr and npages gets set here

### Who calls as_define_region?
`loadelf` calls `as_define_region`. Where does it know how to set the addresses?
* some of the addresses are set in the executable

# stack
The stack is always at a fixed location.

It is created in `as_define_stack`.

# Virtual regions

This is how they usually define it:

```
OS Reserved
Stack (0x7ffffff), growing downward
...
Heap
Unintialized vars (BSS segment)
Initialized vars (data segment)
Code (text segment)
```


### Region implementation notes
* Stack needs to be implemented to assign pages as it growns downward.
    * We need to check to make sure our stack hasn't collided with anything else
* The heap is explicitly extended by C library through `sbrk()` syscall.
    * We need to make sure our heap doesn't smash our stack
The bottom stuff
* determined by exe
* can include shared libs
* set up by linker/loader/OS. OS161 uses `load_elf`

## as_prepare_load
* gets a bunch of physical pages (`getppages()`)
* the number of pages was set earlier from `as_define_region()`
* For DUMBVM: stack is predefined, about 18 pages (72kB)
    * This is terrible. You might run out of contiguous memory
    * Modern solution is paging

## Page tables
* Allocate physical pages *as needed*: one physical page for each virtual page.
* Record virtual-to-physical mapping in a data structure of the address space.
    * this structure is called the **page table**

> **TODO: Page table: virtual page to physical page mapping**

### Translation
What do you do if you do a virtual-physical lookup that's not found in your pagetable!
* **Scenario 1**: virtual address does not belong to any valid region
    * `kill_curthread`: return error from vm_fault
* **Scenario 2**: valid virtual address belongs to a valid virtual region, no physical address exists yet
    * allocate a physical page (page fault)
* **Scenario 3**: valid virtual page belongs to valid virtual region, USED to be a translation, physical page no longer in memory (*paged out*)
    * read the page from disk

# Handling TLB fault
1. Trap the kernel
    * `vm_fault(faultaddress)`, this is what we write
2. Look up corresponding physical page
3. Install the translation into TLB

On MIPS, it makes no sense to get a Page Fault without getting a TLB Fault. How do you get something in the TLB without having it in memory? What the hell?

# Managing physical memory
1. Turn on computer. Nothing is in RAM
2. Exception handlers get loaded at address 0x0
    * IT IS IMPORTANT NOT TO WRITE ANYTHING INTO THE 0x0 PHYSICAL PAGE. THIS IS WHERE INTERRUPT HANDLERS LIVE
    * Advised to put in a bunch of asserts to make sure this never happens
3. Kernel data structures
4. Pages backing user virtual addresses
5. etc. kernel/user data building up

### Freeing memory
DUMBVM's `free_kpages()` does nothing. It just leaks memory.


# Getting started
* Plan data structures
    * Page Table - virtual-to-physical translations
    * Coremap - tracks free and occupied physical memory
* Change kernel to allocate physical pages for virtual regions page-by-page as they get requested from vm_fault
    * store translations in pagetable
    * track allocated pages in coremap
* Add code to properly free memory

> Best bet: do this by Monday

